In [2]:
import pandas as pd
import numpy as np

#import bokeh
from bokeh.plotting import figure, show, Figure
from bokeh.models import ColumnDataSource, Label
from bokeh.models.glyphs import Text
from bokeh.palettes import Spectral3
from bokeh.layouts import row, column, gridplot


# tell bokeh to show the figures in the notebook
from bokeh.io import output_notebook
output_notebook()
from bokeh.plotting import figure, show, output_file

Loading BokehJS ...

In [3]:
filename1 = "DWH_Training.csv"
df1 = pd.read_csv( filename1, header=-1 )
df1.columns=['index','height','weight','gender']
df1.head()
filename2 = "DWH_Test.csv"
df2 = pd.read_csv( filename2, header=-1 )
df2.columns=['index','height','weight','gender','distance']
df2.head()

,index,height,weight,gender,distance
0,1,174.731425,68.346183,-1,4.055077
1,2,167.961304,61.551896,1,-3.245166
2,3,167.291562,69.278931,1,-3.284287
3,4,169.423971,58.349019,1,-2.047819
4,5,175.868380,69.440505,-1,5.277262


In [8]:
male=df1.loc[df1['gender'] == 1]  
female=df1.loc[df1['gender'] == -1]

m_height=male['height']
m_weight=male['weight']
f_height=female['height']
f_weight=female['weight']

def Centroid (m,f): #function call below
    centroidMale = m.mean()
    centroidFemale = f.mean()
    cM = [centroidMale['height'], centroidMale['weight']]
    cF = [centroidFemale['height'], centroidFemale['weight']]
    return cM, cF

centroidMale, centroidFemale = Centroid(male,female) #function call

print('CentMale =', centroidMale)
print('CentFemale =', centroidFemale)

w = [2*centroidMale[i]-2*centroidFemale[i] for i in range(len(centroidMale))]
print('w = ', w)

b = np.linalg.norm(centroidFemale)**2-np.linalg.norm(centroidMale)**2
print('b =', b)

slope = -(w[0]/w[1])
intercept = -(b/w[1])
x = np.linspace(162, 178, num=2)
ncc = [slope*i + intercept for i in x]
#print('ncc = ', ncc)

# for accuracy when test data is encountered

#print(df2.shape) #prints the column x row of test file
predict= np.zeros(df2.shape[0])#assigns zero to all 45 rows of test file and copies it into predict
# print(predict)
result = 0

for index, row in df2.iterrows(): #for loop for iterating through each row of test file
    distance = (w[0] * row['height'] + w[1] * row['weight'] + b) / (np.linalg.norm(w))
 #  print(index)
 #  print(row['height'])
  #  print(distance)
    
    if distance < 0 :      #classify as female
        predict[index] = -1
        if row['gender'] == predict[index]:  #check if value in predict is = to value in original file
            result += 1
    else:               #classify as male
        predict[index] = 1    
        if row['gender'] == predict[index]:  #check if value in predict is = to value in original file
            result += 1
print(result)    
print(df2.shape[0])
print("Accuracy Hyperplane = ", (result/df2.shape[0]) * 100 ,"%")


p = figure(plot_width=600, plot_height=600)
p.xaxis.axis_label = 'Height (cm)'
p.yaxis.axis_label = 'Weight (kg)'
p.circle(m_height, m_weight, size=10, color="green", alpha=0.5,legend="Male",name="Male")
p.circle(f_height, f_weight, size=10, color="red", alpha=0.5,legend="Female",name="Female")
p.line(x,ncc,color='black',line_width=2)

show(p)

CentMale = [168.05780511012142, 61.11494621056542]
CentFemale = [173.59740767241038, 69.42055132833181]
w =  [-11.079205124577925, -16.61121023553278]
b = 2976.8103885587443
34
45
Accuracy Hyperplane =  75.55555555555556 %
